In [12]:
import json
import numpy as np
from collections import defaultdict
from collections import OrderedDict

In [55]:
f = open('prediction/all_boxscores_2000_2017.json')
boxscore_data = json.load(f)

In [56]:
boxscore_data_ordered_by_key = OrderedDict(sorted(boxscore_data.items()))

In [58]:
# build running averages of stats in a season for a team
# team-ID -> season -> team's game num -> stat_type -> average (inclusive of current game)
team_boxscore_rolling_averages = defaultdict(lambda: defaultdict(lambda: defaultdict(dict)))

boxscore_categories = ['boxscoretraditionalv2', 'boxscoreadvancedv2', 'boxscoremiscv2', 'boxscorescoringv2',
                       'boxscoreusagev2', 'boxscorefourfactorsv2', 'boxscoreplayertrackv2', 'hustlestatsboxscore',
                       'boxscoresummaryv2']

BOXSCORE_TRADITIONAL_FEATURES = []

def prune_team_stats(team_stats):
  pruned_team_stats = []
  stat_name_index_tuples = [
    ('FGM', 6),
    ('FGA', 7),
    ('FG_PCT', 8),
    ('FG3M', 9),
    ('FG3A', 10),
    ('FG3_PCT', 11),
    ('FTM', 12),
    ('FTA', 13),
    ('FT_PCT', 14),
    ('OREB', 15),
    ('DREB', 16),
    ('REB', 17),
    ('AST', 18),
    ('STL', 19),
    ('BLK', 20),
    ('TO', 21),
    ('PF', 22),
    ('PTS', 23),
    ('PLUS_MINUS', 24)
  ]
  
  for stat_name_index_tuple in stat_name_index_tuples:
    pruned_team_stats.append(team_stats[stat_name_index_tuple[1]])
  
  return np.asarray(pruned_team_stats)

def prune_starter_bench_stats(team_starter_bench_stats):
  pruned_team_starter_bench_stats = []
  stat_name_index_tuples = [
    ('FGM', 7),
    ('FGA', 8),
    ('FG_PCT', 9),
    ('FG3M', 10),
    ('FG3A', 11),
    ('FG3_PCT', 12),
    ('FTM', 13),
    ('FTA', 14),
    ('FT_PCT', 15),
    ('OREB', 16),
    ('DREB', 17),
    ('REB', 18),
    ('AST', 19),
    ('STL', 20),
    ('BLK', 21),
    ('TO', 22),
    ('PF', 23),
    ('PTS', 24),
  ]
  
  for stat_name_index_tuple in stat_name_index_tuples:
    pruned_team_starter_bench_stats.append(team_starter_bench_stats[stat_name_index_tuple[1]])
  
  return np.asarray(pruned_team_starter_bench_stats)

# maintains state about how many games were played by a team in a season, reset every season
# TEAM_ID -> num games played
team_games_played_dict = defaultdict(lambda: 1)

prev_game_num = None
for json_file_name, data in boxscore_data_ordered_by_key.iteritems():
  game_id = json_file_name.split('.')[0]
  print game_id
  game_num = game_id[-4:]
  season = game_id[3:5]
  print game_num
  if game_num == '0001':
    print "got here"
    team_games_played_dict = defaultdict(lambda: 1)
  for boxscore_category in boxscore_categories:
    category_data = boxscore_data[json_file_name][boxscore_category]
    if boxscore_category == 'boxscoretraditionalv2':
      if 'resultSets' in category_data:
        data = category_data['resultSets']
        # first team in list is away, second is home
        if len(data[1]['rowSet']) == 0:
          continue
        away_team_id = data[1]['rowSet'][0][1]
        home_team_id = data[1]['rowSet'][1][1]
        team_stats_away = data[1]['rowSet'][0]
        team_stats_home = data[1]['rowSet'][1]
        pruned_team_stats_away_features = prune_team_stats(team_stats_away)
        pruned_team_stats_home_features = prune_team_stats(team_stats_home)
        team_starter_stats_away = data[2]['rowSet'][0]
        team_bench_stats_away = data[2]['rowSet'][1]
        team_starter_stats_home = data[2]['rowSet'][2]
        team_bench_stats_home = data[2]['rowSet'][3]
        pruned_starter_stats_away = prune_starter_bench_stats(team_starter_stats_away)
        pruned_bench_stats_away = prune_starter_bench_stats(team_bench_stats_away)
        pruned_starter_stats_home = prune_starter_bench_stats(team_starter_stats_home)
        pruned_bench_stats_home = prune_starter_bench_stats(team_bench_stats_away)
        
        if team_games_played_dict[away_team_id] == 1:
          team_boxscore_rolling_averages[away_team_id][season][1]['team_stats'] = pruned_team_stats_away_features
        else:
          prev_game_num = team_games_played_dict[away_team_id] - 1
          curr_game_num = team_games_played_dict[away_team_id]
          prev_away_team_stats = team_boxscore_rolling_averages[away_team_id][season][prev_game_num]['team_stats']
          team_boxscore_rolling_averages[away_team_id][season][curr_game_num]['team_stats'] = \
            prev_away_team_stats + \
            (pruned_team_stats_away_features - prev_away_team_stats)/curr_game_num
        
        team_games_played_dict[away_team_id] += 1

        if team_games_played_dict[home_team_id] == 1:
          team_boxscore_rolling_averages[home_team_id][season][1]['team_stats'] = pruned_team_stats_home_features
        else:
          prev_game_num = team_games_played_dict[home_team_id] - 1
          curr_game_num = team_games_played_dict[home_team_id]
          prev_home_team_stats = team_boxscore_rolling_averages[home_team_id][season][prev_game_num]['team_stats']
          team_boxscore_rolling_averages[home_team_id][season][curr_game_num]['team_stats'] = \
            prev_home_team_stats + \
            (pruned_team_stats_home_features - prev_home_team_stats)/curr_game_num
        
        team_games_played_dict[home_team_id] += 1

    else:
      # ignore other categories for now
      break


# generate features
features = ['HOME_PTS_PAINT', 'AWAY_PTS_PAINT', 'HOME_PTS_2ND_CHANCE', 'AWAY_PTS_2ND_CHANCE', 
            'HOME_PTS_FB', 'AWAY_PTS_FB', 'HOME_LARGEST_LEAD', 'AWAY_LARGEST_LEAD', 
            'LEAD_CHANGES', 'TIMES_TIED', 'HOME_TURNOVERS', 'AWAY_TURNOVERS', 
            'HOME_PTS_OFF_TO', 'AWAY_PTS_OFF_TO']

0020000001
0001
got here
0020000002
0002
0020000003
0003
0020000004
0004
0020000005
0005
0020000006
0006
0020000007
0007
0020000008
0008
0020000009
0009
0020000010
0010
0020000011
0011
0020000012
0012
0020000013
0013
0020000014
0014
0020000015
0015
0020000016
0016
0020000017
0017
0020000018
0018
0020000019
0019
0020000020
0020
0020000021
0021
0020000022
0022
0020000023
0023
0020000024
0024
0020000025
0025
0020000026
0026
0020000027
0027
0020000028
0028
0020000029
0029
0020000030
0030
0020000031
0031
0020000032
0032
0020000033
0033
0020000034
0034
0020000035
0035
0020000036
0036
0020000037
0037
0020000038
0038
0020000039
0039
0020000040
0040
0020000041
0041
0020000042
0042
0020000043
0043
0020000044
0044
0020000045
0045
0020000046
0046
0020000047
0047
0020000048
0048
0020000049
0049
0020000050
0050
0020000051
0051
0020000052
0052
0020000053
0053
0020000054
0054
0020000055
0055
0020000056
0056
0020000057
0057
0020000058
0058
0020000059
0059
0020000060
0060
0020000061
0061
0020000062
0062


0020000099
0099
0020000100
0100
0020000101
0101
0020000102
0102
0020000103
0103
0020000104
0104
0020000105
0105
0020000106
0106
0020000107
0107
0020000108
0108
0020000109
0109
0020000110
0110
0020000111
0111
0020000112
0112
0020000113
0113
0020000114
0114
0020000115
0115
0020000116
0116
0020000117
0117
0020000118
0118
0020000119
0119
0020000120
0120
0020000121
0121
0020000122
0122
0020000123
0123
0020000124
0124
0020000125
0125
0020000126
0126
0020000127
0127
0020000128
0128
0020000129
0129
0020000130
0130
0020000131
0131
0020000132
0132
0020000133
0133
0020000134
0134
0020000135
0135
0020000136
0136
0020000137
0137
0020000138
0138
0020000139
0139
0020000140
0140
0020000141
0141
0020000142
0142
0020000143
0143
0020000144
0144
0020000145
0145
0020000146
0146
0020000147
0147
0020000148
0148
0020000149
0149
0020000150
0150
0020000151
0151
0020000152
0152
0020000153
0153
0020000154
0154
0020000155
0155
0020000156
0156
0020000157
0157
0020000158
0158
0020000159
0159
0020000160
0160
0020000


0020000274
0274
0020000275
0275
0020000276
0276
0020000277
0277
0020000278
0278
0020000279
0279
0020000280
0280
0020000281
0281
0020000282
0282
0020000283
0283
0020000284
0284
0020000285
0285
0020000286
0286
0020000287
0287
0020000288
0288
0020000289
0289
0020000290
0290
0020000291
0291
0020000292
0292
0020000293
0293
0020000294
0294
0020000295
0295
0020000296
0296
0020000297
0297
0020000298
0298
0020000299
0299
0020000300
0300
0020000301
0301
0020000302
0302
0020000303
0303
0020000304
0304
0020000305
0305
0020000306
0306
0020000307
0307
0020000308
0308
0020000309
0309
0020000310
0310
0020000311
0311
0020000312
0312
0020000313
0313
0020000314
0314
0020000315
0315
0020000316
0316
0020000317
0317
0020000318
0318
0020000319
0319
0020000320
0320
0020000321
0321
0020000322
0322
0020000323
0323
0020000324
0324
0020000325
0325
0020000326
0326
0020000327
0327
0020000328
0328
0020000329
0329
0020000330
0330
0020000331
0331
0020000332
0332
0020000333
0333
0020000334
0334
0020000335
0335
0020000

0020000397
0397
0020000398
0398
0020000399
0399
0020000400
0400
0020000401
0401
0020000402
0402
0020000403
0403
0020000404
0404
0020000405
0405
0020000406
0406
0020000407
0407
0020000408
0408
0020000409
0409
0020000410
0410
0020000411
0411
0020000412
0412
0020000413
0413
0020000414
0414
0020000415
0415
0020000416
0416
0020000417
0417
0020000418
0418
0020000419
0419
0020000420
0420
0020000421
0421
0020000422
0422
0020000423
0423
0020000424
0424
0020000425
0425
0020000426
0426
0020000427
0427
0020000428
0428
0020000429
0429
0020000430
0430
0020000431
0431
0020000432
0432
0020000433
0433
0020000434
0434
0020000435
0435
0020000436
0436
0020000437
0437
0020000438
0438
0020000439
0439
0020000440
0440
0020000441
0441
0020000442
0442
0020000443
0443
0020000444
0444
0020000445
0445
0020000446
0446
0020000447
0447
0020000448
0448
0020000449
0449
0020000450
0450
0020000451
0451
0020000452
0452
0020000453
0453
0020000454
0454
0020000455
0455
0020000456
0456
0020000457
0457
0020000458
0458
00200004

0020000501
0501
0020000502
0502
0020000503
0503
0020000504
0504
0020000505
0505
0020000506
0506
0020000507
0507
0020000508
0508
0020000509
0509
0020000510
0510
0020000511
0511
0020000512
0512
0020000513
0513
0020000514
0514
0020000515
0515
0020000516
0516
0020000517
0517
0020000518
0518
0020000519
0519
0020000520
0520
0020000521
0521
0020000522
0522
0020000523
0523
0020000524
0524
0020000525
0525
0020000526
0526
0020000527
0527
0020000528
0528
0020000529
0529
0020000530
0530
0020000531
0531
0020000532
0532
0020000533
0533
0020000534
0534
0020000535
0535
0020000536
0536
0020000537
0537
0020000538
0538
0020000539
0539
0020000540
0540
0020000541
0541
0020000542
0542
0020000543
0543
0020000544
0544
0020000545
0545
0020000546
0546
0020000547
0547
0020000548
0548
0020000549
0549
0020000550
0550
0020000551
0551
0020000552
0552
0020000553
0553
0020000554
0554
0020000555
0555
0020000556
0556
0020000557
0557
0020000558
0558
0020000559
0559
0020000560
0560
0020000561
0561
0020000562
0562
00200005

...

0021600244
0244
0021600245
0245
0021600246
0246
0021600247
0247
0021600248
0248
0021600249
0249
0021600250
0250
0021600251
0251
0021600252
0252
0021600253
0253
0021600254
0254
0021600255
0255
0021600256
0256
0021600257
0257
0021600258
0258
0021600259
0259
0021600260
0260
0021600261
0261
0021600262
0262
0021600263
0263
0021600264
0264
0021600265
0265
0021600266
0266
0021600267
0267
0021600268
0268
0021600269
0269
0021600270
0270
0021600271
0271
0021600272
0272
0021600273
0273
0021600274
0274
0021600275
0275
0021600276
0276
0021600277
0277
0021600278
0278
0021600279
0279
0021600280
0280
0021600281
0281
0021600282
0282
0021600283
0283
0021600284
0284
0021600285
0285
0021600286
0286
0021600287
0287
0021600288
0288
0021600289
0289
0021600290
0290
0021600291
0291
0021600292
0292
0021600293
0293
0021600294
0294
0021600295
0295
0021600296
0296
0021600297
0297
0021600298
0298
0021600299
0299
0021600300
0300
0021600301
0301
0021600302
0302
0021600303
0303
0021600304
0304
0021600305
0305
00216003

0453
0021600454
0454
0021600455
0455
0021600456
0456
0021600457
0457
0021600458
0458
0021600459
0459
0021600460
0460
0021600461
0461
0021600462
0462
0021600463
0463
0021600464
0464
0021600465
0465
0021600466
0466
0021600467
0467
0021600468
0468
0021600469
0469
0021600470
0470
0021600471
0471
0021600472
0472
0021600473
0473
0021600474
0474
0021600475
0475
0021600476
0476
0021600477
0477
0021600478
0478
0021600479
0479
0021600480
0480
0021600481
0481
0021600482
0482
0021600483
0483
0021600484
0484
0021600485
0485
0021600486
0486
0021600487
0487
0021600488
0488
0021600489
0489
0021600490
0490
0021600491
0491
0021600492
0492
0021600493
0493
0021600494
0494
0021600495
0495
0021600496
0496
0021600497
0497
0021600498
0498
0021600499
0499
0021600500
0500
0021600501
0501
0021600502
0502
0021600503
0503
0021600504
0504
0021600505
0505
0021600506
0506
0021600507
0507
0021600508
0508
0021600509
0509
0021600510
0510
0021600511
0511
0021600512
0512
0021600513
0513
0021600514
0514
0021600515
0515
002


0631
0021600632
0632
0021600633
0633
0021600634
0634
0021600635
0635
0021600636
0636
0021600637
0637
0021600638
0638
0021600639
0639
0021600640
0640
0021600641
0641
0021600642
0642
0021600643
0643
0021600644
0644
0021600645
0645
0021600646
0646
0021600647
0647
0021600648
0648
0021600649
0649
0021600650
0650
0021600651
0651
0021600652
0652
0021600653
0653
0021600654
0654
0021600655
0655
0021600656
0656
0021600657
0657
0021600658
0658
0021600659
0659
0021600660
0660
0021600661
0661
0021600662
0662
0021600663
0663
0021600664
0664
0021600665
0665
0021600666
0666
0021600667
0667
0021600668
0668
0021600669
0669
0021600670
0670
0021600671
0671
0021600672
0672
0021600673
0673
0021600674
0674
0021600675
0675
0021600676
0676
0021600677
0677
0021600678
0678
0021600679
0679
0021600680
0680
0021600681
0681
0021600682
0682
0021600683
0683
0021600684
0684
0021600685
0685
0021600686
0686
0021600687
0687
0021600688
0688
0021600689
0689
0021600690
0690
0021600691
0691
0021600692
0692
0021600693
0693
00

0021600755
0755
0021600756
0756
0021600757
0757
0021600758
0758
0021600759
0759
0021600760
0760
0021600761
0761
0021600762
0762
0021600763
0763
0021600764
0764
0021600765
0765
0021600766
0766
0021600767
0767
0021600768
0768
0021600769
0769
0021600770
0770
0021600771
0771
0021600772
0772
0021600773
0773
0021600774
0774
0021600775
0775
0021600776
0776
0021600777
0777
0021600778
0778
0021600779
0779
0021600780
0780
0021600781
0781
0021600782
0782
0021600783
0783
0021600784
0784
0021600785
0785
0021600786
0786
0021600787
0787
0021600788
0788
0021600789
0789
0021600790
0790
0021600791
0791
0021600792
0792
0021600793
0793
0021600794
0794
0021600795
0795
0021600796
0796
0021600797
0797
0021600798
0798
0021600799
0799
0021600800
0800
0021600801
0801
0021600802
0802
0021600803
0803
0021600804
0804
0021600805
0805
0021600806
0806
0021600807
0807
0021600808
0808
0021600809
0809
0021600810
0810
0021600811
0811
0021600812
0812
0021600813
0813
0021600814
0814
0021600815
0815
0021600816
0816
00216008


0021601102
1102
0021601103
1103
0021601104
1104
0021601105
1105
0021601106
1106
0021601107
1107
0021601108
1108
0021601109
1109
0021601110
1110
0021601111
1111
0021601112
1112
0021601113
1113
0021601114
1114
0021601115
1115
0021601116
1116
0021601117
1117
0021601118
1118
0021601119
1119
0021601120
1120
0021601121
1121
0021601122
1122
0021601123
1123
0021601124
1124
0021601125
1125
0021601126
1126
0021601127
1127
0021601128
1128
0021601129
1129
0021601130
1130
0021601131
1131
0021601132
1132
0021601133
1133
0021601134
1134
0021601135
1135
0021601136
1136
0021601137
1137
0021601138
1138
0021601139
1139
0021601140
1140
0021601141
1141
0021601142
1142
0021601143
1143
0021601144
1144
0021601145
1145
0021601146
1146
0021601147
1147
0021601148
1148
0021601149
1149
0021601150
1150
0021601151
1151
0021601152
1152
0021601153
1153
0021601154
1154
0021601155
1155
0021601156
1156
0021601157
1157
0021601158
1158
0021601159
1159
0021601160
1160
0021601161
1161
0021601162
1162
0021601163
1163
0021601

In [62]:
team_boxscore_rolling_averages[1610612760]['16']

defaultdict(dict,
            {1: {'team_stats': array([  39.   ,   94.   ,    0.415,    6.   ,   22.   ,    0.273,
                       19.   ,   26.   ,    0.731,   13.   ,   40.   ,   53.   ,
                       19.   ,    7.   ,    3.   ,   10.   ,   23.   ,  103.   ,    6.   ])},
             2: {'team_stats': array([  39.5   ,   95.    ,    0.416 ,    5.5   ,   21.5   ,    0.2555,
                       23.5   ,   32.    ,    0.734 ,   13.5   ,   41.5   ,   55.    ,
                       16.    ,    9.    ,    3.    ,   16.    ,   23.5   ,  108.    ,
                        4.5   ])},
             3: {'team_stats': array([  41.33333333,   95.        ,    0.43533333,    6.66666667,
                       23.        ,    0.28566667,   20.33333333,   27.        ,
                        0.764     ,   12.33333333,   39.33333333,   51.66666667,
                       20.33333333,   11.        ,    4.        ,   17.        ,
                       23.        ,  109.66666667,    8